# Setup

In [41]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Path('../backend/shuscribe').resolve()
sys.path.insert(0, str(Path('../backend').resolve()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
# Import necessary modules
import asyncio
from dotenv import load_dotenv
import os
from shuscribe.services.llm.session import LLMSession
from shuscribe.services.llm.providers.provider import (
    Message, GenerationConfig
)
from shuscribe.schemas.llm import MessageRole

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
ANTHROPIC_API_KEY = os.environ["ANTHROPIC_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

TEST_MODELS ={
    "openai": "gpt-4o-mini",
    "anthropic": "claude-3-5-haiku-20241022",
    "gemini": "gemini-2.0-flash-001"
}

TEST_THINKING_MODELS = {
    "openai": "o3-mini-2025-01-31",
    "anthropic": "claude-3-7-sonnet-20250219",
    "gemini": "gemini-2.0-flash-thinking-exp"
}

# Helper function to run async code in notebook
async def run_async(coro):
    return await coro

In [43]:
# Streaming response
async def stream(provider_name: str, messages: list[Message]):
    async with LLMSession.session_scope() as session:
        # Create a streaming config
        config = GenerationConfig(temperature=0.7)
        
        print(f"{TEST_MODELS[provider_name]}:")

        async for chunk in session.generate_stream(
            messages=messages,
            provider_name=provider_name,
            model=TEST_MODELS[provider_name],
            config=config
        ):
            print(chunk.text, end="", flush=True)


In [44]:
import yaml

from shuscribe.schemas.pipeline import Chapter

with open("../backend/tests/resources/pokemon_amber/_meta.yaml", "r") as f:
    meta = yaml.safe_load(f)

STORY_TITLE = meta.get('story_title')
STORY_DESCRIPTION = meta.get('story_description')
GENRES = meta.get('genres')
ADDITIONAL_TAGS = meta.get('additional_tags')

CHAPTERS = []
for chapter in meta.get('chapters'):
    with open(f"../backend/tests/resources/pokemon_amber/{chapter}", "r") as f:
        chapter_id = chapter.split('.')[0]
        try:
            chapter_content = yaml.safe_load(f)
            CHAPTERS.append(Chapter(id=chapter_id, title=chapter_content.get('title'), content=chapter_content.get('content')))
        except Exception:
            # print(f"Error loading chapter {chapter}: {e}")
            continue


# Chapter Summary

In [45]:
# Set Provider Name
PROVIDER_NAME = "gemini"

In [46]:
from shuscribe.services.llm.prompts.manager import PromptManager

prompt_manager = PromptManager()

messages: list[Message] = prompt_manager.chapter.summary(
    current_chapter=CHAPTERS[0],
)

messages

[Message(role=<MessageRole.SYSTEM: 'system'>, content='You are an expert Narrative Knowledge Assistant, combining the skills of a literary analyst, teacher, and storytelling guide.', name=None),
 Message(role=<MessageRole.USER: 'user'>, content='# Overarching Goal\nExtract and structure knowledge from the narrative while maintaining proper story progression context and preventing spoilers.\n\n# Narrative Context\n\n## Current Chapter\n### Chapter 1: [Chapter 1] Truck-kun Strikes Again\n\n<Chapter>\n\'Fuck,\' I thought, hunched over my phone in the fading afternoon light. My fingers hovered over the screen as if I could somehow intimidate the numbers into change.\nMy Gyarados—level 40, carefully trained, survivor of multiple gym battles and approximately eight thousand random Pidgey and Mankey encounters—was down to a sliver of health. Across the battlefield, a level 37 Vulpix stared back with pixels that somehow managed to look smug. It had probably practiced that look in the mirror.\n

In [47]:
streaming_response = await run_async(stream(PROVIDER_NAME, messages))

gemini-2.0-flash-001:
Okay, I'm ready to read the chapter and provide a detailed, progressive summary, pointing out literary elements as I go.

<|STARTOFSUMMARY|>
# Summary

**Scene 1: A Gamer's Dilemma**

The chapter opens with the protagonist, AlexaTheGreat, deeply engrossed in a Pokémon game on an emulator. The stakes are high due to perma-death rules, and she's facing a challenging battle in the Pokemon Mansion. The description of her carefully trained Gyarados being on its last sliver of health immediately establishes tension and highlights the dedication she has to the game. The mention of past battles and the fear of losing further Pokémon emphasizes the high-stakes nature of her gaming experience. The setting is her dorm room, contrasting the mundane reality of campus life with the intense virtual world she inhabits. This contrast foreshadows the blurring of reality and the virtual that will occur later in the chapter. The internal monologue reveals her strategic thinking and t

Database module not implemented. Skipping save.
